In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import bz2
import json
from tld import get_tld
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
import math
from collections import Counter
from operator import itemgetter
import re
import pickle

nltk.download()
nltk.download('stopwords')
nltk.download('punkt')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Arthur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Arthur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from Data_clean_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Arthur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Arthur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### CREATE PATHS: change according to local file

In [3]:
DATA_PATH = 'Data/'
FILE = DATA_PATH + 'quotes-2020.json.bz2'
PATH_OUT = DATA_PATH + 'clean-quotes-2020.json.bz2'
#PATH_OUT_TEST = DATA_PATH + 'clean-quotes-2020_TEST.json.bz2'

In [4]:
df_base = pd.read_json(FILE, lines=True, compression='bz2', nrows=1000)
df_base

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E
1,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
2,2020-02-10-000142,... He (Madhav) also disclosed that the illega...,None,[],2020-02-10 23:45:54,1,"[[None, 0.8926], [Prakash Rai, 0.1074]]",[https://indianexpress.com/article/business/ec...,E
3,2020-02-15-000053,"... [ I ] f it gets to the floor,",None,[],2020-02-15 14:12:51,2,"[[None, 0.581], [Andy Harris, 0.4191]]",[https://patriotpost.us/opinion/68622-trump-bu...,E
4,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
...,...,...,...,...,...,...,...,...,...
995,2020-01-12-021411,is like if we played for the national champion...,Dabo Swinney,[Q5207523],2020-01-12 12:00:00,1,"[[Dabo Swinney, 0.9678], [None, 0.0309], [Pete...",[http://postandcourier.com/sports/clemson/sapa...,E
996,2020-01-21-042452,is part of the ongoing transformation of Eaton...,Craig Arnold,[Q5180704],2020-01-21 16:54:50,2,"[[Craig Arnold, 0.8338], [None, 0.1662]]",[http://crainscleveland.com/manufacturing/eato...,E
997,2020-02-21-033839,"Is that a genuine one? Here, we'll swap so,",Seamus Hughes,[Q7665862],2020-02-21 11:00:00,1,"[[Seamus Hughes, 0.7277], [None, 0.1458], [Mar...",[https://www.longfordleader.ie/news/local-news...,E
998,2020-02-21-033859,Is the kid doing it?,William Brent Bell,[Q8005891],2020-02-21 07:37:55,2,"[[William Brent Bell, 0.8468], [None, 0.1532]]",[http://www.breitbart.com/news/jared-kushner-i...,E


In [5]:
Wikidata_speakers = pd.read_parquet(DATA_PATH + 'speaker_attributes.parquet')
Wikidata_speakers

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[Q30],[Q6581097],1397399351,None,None,[Q82955],None,None,Q106406560,Barker B. Howard,None,item,None
9055977,[Charles Macomber],None,[Q30],[Q6581097],1397399471,None,None,[Q82955],None,None,Q106406571,Charles H. Macomber,None,item,None
9055978,None,[+1848-04-01T00:00:00Z],None,[Q6581072],1397399751,None,None,None,None,None,Q106406588,Dina David,None,item,None
9055979,None,[+1899-03-18T00:00:00Z],None,[Q6581072],1397399799,None,None,None,None,None,Q106406593,Irma Dexinger,None,item,None


In [6]:
Wikidata_countries = pd.read_csv(DATA_PATH + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression = 'bz2')
Wikidata_countries

,QID,Label,Description
0,Q31,Belgium,country in western Europe
1,Q45,Portugal,country in southwestern Europe
2,Q75,Internet,global system of connected computer networks
3,Q148,People's Republic of China,sovereign state in East Asia
4,Q155,Brazil,country in South America
...,...,...,...
36964,Q106302506,didgeridooist,musician who plays the didgeridoo
36965,Q106341153,biochemistry teacher,teacher of biochemistry at any level
36966,Q106368830,2018 Wigan Metropolitan Borough Council electi...,NaN
36967,Q106369692,2018 Wigan Metropolitan Borough Council electi...,NaN


## Create a dictionnary of categories and associated synonyms

In [7]:
matchers = {"art": ["art", "paint", "draw", "museum"], \
            "business": ["business", "finance", "economy", "commerce", "bank", "money", "trade"], \
            "entertainment":["entertainment"], 
            "fashion":["fashion", "couture", "designer"], \
            "medicine":["medicine", "health", "pharmacy", "wellbeing", "body"], \
            "music":["music", "song", "album", "concert"], \
            "politics":["politics", "government"], \
            "science":["science", "research"], \
            "sport": ["sport", "football", "athletics", "swimming", "rugby", "tennis", "volleyball", "ski"]}

In [ ]:
# Don't run this as doesn't return good list

'''

def find_synonyms(matchers):
    for category in matchers:
        synonyms = []
        for i in range(len(matchers[category])):
            word = matchers[category][i]
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    synonyms.append(l.name())
            matchers[category] = synonyms
    return matchers

matchers = find_synonyms(matchers)

'''

In [8]:
generalizeDictionary(matchers)

In [9]:
matchers

{'art': ['art', 'paint', 'draw', 'museum'],
 'business': ['busi',
  'financ',
  'economi',
  'commerc',
  'bank',
  'money',
  'trade'],
 'entertainment': ['entertain'],
 'fashion': ['fashion', 'coutur', 'design'],
 'medicine': ['medicin', 'health', 'pharmaci', 'wellb', 'bodi'],
 'music': ['music', 'song', 'album', 'concert'],
 'politics': ['polit', 'govern'],
 'science': ['scienc', 'research'],
 'sport': ['sport',
  'footbal',
  'athlet',
  'swim',
  'rugbi',
  'tenni',
  'volleybal',
  'ski']}

### PROCESSING DATA

In [ ]:
# Pickle the chunks of dataframe for later
n_chunks = 0

with pd.read_json(FILE, lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers, Wikidata_speakers, Wikidata_countries)
        with open(PATH_OUT, 'ab') as d_file:
            pickle.dump(chunk_cleaned, d_file) 
        n_chunks += 1

Processing chunk with 1000 rows


In [ ]:
# Open pickled file
chunks_all = pd.DataFrame(columns=['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'gender', 'citizenship', 'Description',
       'sitenames', 'domain', 'tags'])
chunk_nbr = 0
with open(PATH_OUT, 'rb') as d_file:
    while (chunk_nbr < n_chunks):
        chunk = pickle.load(d_file)
        chunks_all = chunks_all.append(chunk)
        chunk_nbr+=1
        print(chunk_nbr)
        # Need to load m times if you want to get to the m^th file

In [ ]:
chunks_all

### Without pickle / writing to new json

In [ ]:
with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    with bz2.open(PATH_OUT, 'wb') as d_file:
        for chunk in df_reader:
            chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers, Wikidata_speakers, Wikidata_countries)
            chunk_json = chunk_cleaned.to_json(orient='columns')#, index=False) #write it to pickle file instead
            d_file.write((chunk_json+'\n').encode('utf-8'))

In [ ]:
chunk_cleaned

In [ ]:
df = pd.read_json(PATH_OUT, orient='columns', compression='bz2')
df

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2', lines=True, nrows=100)
df

In [ ]:
with pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    
